# Clinical Data De-Identification and Time Shifting

This notebook performs data de-identification and time shifting on clinical datasets to protect patient privacy while maintaining data utility for research purposes. The process:

1. Encrypts patient identifiers using a cryptographic hash with a random salt
2. Applies a consistent time shift to all dates related to the same patient
3. Processes multiple data tables while maintaining referential integrity
4. Removes directly identifiable information

## Setup and Imports
Import necessary libraries and establish database connection.

In [17]:
# imports per postgres del pzero

import json
from sshtunnel import SSHTunnelForwarder
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import sys
from sqlalchemy import inspect
import pandas as pd
import os
import json
import sys

ROOT_PATH = "/Users/xaviborrat/Documents/GitHub/TFM_Clinical_Deterioration"

sys.path.append(ROOT_PATH)
#importació llibreria de classes de connexio
import classes
from classes.xavi_con_class import db_connect as xcc





## Database Connection
Establish connection to the database and generate a random cryptographic salt for consistent encryption.

In [ ]:

# Connector class instatiation using config_tfm.json

config_path = os.path.join(ROOT_PATH, 'classes/config_tfm.json')
with open(config_path, 'r') as configDnx:
    configDnx = json.load(configDnx)

datanex= xcc(
    ssh = configDnx["ssh"],
    ssh_user = configDnx["ssh_user"],
    ssh_host = configDnx["ssh_host"],
    ssh_pkey = configDnx["ssh_pkey"],
    db_host = configDnx["db_host"],
    db_port = configDnx["db_port"],
    db_user = configDnx["db_user"],
    db_pass = configDnx["db_pass"],
    flavour = configDnx["flavour"],
    db = configDnx["db"]
)


# Random number generator
SALT = os.urandom(32)

## Test Encryption Function
Demonstrate how the encryption function works with a sample value.

In [ ]:
# encryption example

xcc.encrypt_to_x_digits('number=123245647344890',4, SALT)

6655

## Process Laboratory Data
De-identify and time-shift laboratory test data.

In [20]:
# LABS DEID (encriptem labs) i time shift. 5 min

labs = datanex.query(db='datascope4', query='select * from borrat_project.labs')


# time shift utilitzant encriptat de patient_ref. Dintre dels temps de cada pacient mateix shift
labs['weeks_to_add'] = labs['patient_ref'].apply(lambda x: xcc.encrypt_to_x_digits('number='+str(x),3, SALT))
labs['weeks_to_add'] = pd.to_timedelta(labs['weeks_to_add'],unit='W')
labs['extract_date'] = labs['extrac_date']+labs['weeks_to_add']


# eliminació de columnes innecessaries
labs = labs.drop(['patient_ref','weeks_to_add','extrac_date'],axis=1)


# pujada de labs a datanex
datanex.write_table_2(db='borrat_project', df=labs, table_name='labs_ds')

Database [datascope4] session created...
<> Query Sucessful <>
Database [borrat_project] session created...
<> Table [labs_ds] created <>


## Process Ward Stays Data
De-identify and time-shift ward stay data, including admission/discharge dates and patient identifiers.

In [21]:
# WARD_STAYS DEID (encriptem ward_ward_stays) i time shift. 30 seg


# lectura de  ward_stays
#ward_stays = datanex.query(db='datascope4', query='select * from p_zero_phi.ward_ward_stays',parse_dates=['ward_adm_date','ward_disch_date','hosp_mortality_date,icu_los'])
ward_stays = datanex.query(db='datascope4', query='select * from borrat_project.ward_stays ')

demog = datanex.query(db='datascope4', query='select * from borrat_project.demographics',parse_dates=['birth_date'])

# merge de ward_stays amb demog
ward_stays = pd.merge(ward_stays, demog[['patient_ref','birth_date']], on='patient_ref', how='left')



# # deidentifiquem  els ward_stays
ward_stays['patient_id'] = ward_stays['patient_ref'].apply(lambda x: xcc.encrypt_to_x_digits('number='+str(x),10, SALT))
ward_stays['episode_ref'] = ward_stays['episode_ref'].apply(lambda x: xcc.encrypt_to_x_digits('number='+str(x),10, SALT))


#  time shift de setmanes utilitzant encriptat de patient_ref. Dintre dels temps de cada pacient mateix shift
ward_stays['weeks_to_add'] = ward_stays['patient_ref'].apply(lambda x: xcc.encrypt_to_x_digits('number='+str(x),3, SALT))
ward_stays['weeks_to_add'] = pd.to_timedelta(ward_stays['weeks_to_add'],unit='W')

# ward_stays te 5 dates de les quals eliminarem birth_date
ward_stays['start_date'] = ward_stays['start_date']+ward_stays['weeks_to_add']
ward_stays['end_date'] = ward_stays['end_date']+ward_stays['weeks_to_add']
ward_stays['hosp_adm_date'] = ward_stays['hosp_adm_date']+ward_stays['weeks_to_add']
ward_stays['hosp_disch_date'] = ward_stays['hosp_disch_date']+ward_stays['weeks_to_add']
ward_stays['birth_date'] = ward_stays['birth_date']+ward_stays['weeks_to_add']

# calcul de l'edat a l'ingres perque elimarem edat de demografics. 
ward_stays['age_on_admission'] = ward_stays['hosp_adm_date'].apply(lambda x: x.year) - ward_stays['birth_date'].apply(lambda x: x.year)



# # eliminació de columnes innecessaries
ward_stays = ward_stays.drop(['patient_ref','birth_date','weeks_to_add'],axis=1)

# # tornem a posar els noms originals
ward_stays = ward_stays.rename(columns={'patient_id':'patient_ref'})

# # afegim un id per a cada stay per a fer d'ancora per als joins
# ward_stays['stay_id'] = range(1, len(ward_stays) + 1)

# # pujada de labs a datanex
datanex.write_table_2(db='borrat_project', df=ward_stays, table_name='ward_stays_ds')


Database [datascope4] session created...
<> Query Sucessful <>
Database [datascope4] session created...
<> Query Sucessful <>
Database [borrat_project] session created...
<> Table [ward_stays_ds] created <>


## Check Vital Signs Data
Retrieve and examine the vital signs data after processing.

In [22]:
vitals_ds = datanex.query(db='borrat_project', query='select * from borrat_project.vitals_ds')

vitals_ds[vitals_ds.rc_sap_ref == 'O2_DIS']

Database [borrat_project] session created...
<> Query Sucessful <>


,stay_id,result_date,result_num,result_txt,rc_sap_ref
23,5765,2024-10-02 09:00:00,NaN,O2_DIS_1,O2_DIS
55,5765,2024-09-29 09:00:00,NaN,O2_DIS_2,O2_DIS
58,5765,2024-10-03 09:00:00,NaN,O2_DIS_6,O2_DIS
61,5765,2024-10-06 09:00:00,NaN,O2_DIS_6,O2_DIS
81,5765,2024-10-11 23:00:00,NaN,O2_DIS_6,O2_DIS
...,...,...,...,...,...
5992211,2529,2034-06-15 17:51:00,NaN,O2_DIS_6,O2_DIS
5992220,2529,2034-06-19 09:20:00,NaN,O2_DIS_6,O2_DIS
5992227,2529,2034-06-14 09:39:00,NaN,O2_DIS_6,O2_DIS
5992236,2529,2034-06-19 00:05:00,NaN,O2_DIS_6,O2_DIS


## Process Vital Signs Data
De-identify and time-shift vital signs data.

In [23]:
# VITALS (encriptem identificadors) i time shift. 7 min


# lectura de vitals de datanex_phi
vitals = datanex.query(db='borrat_project', query='select * from vitals',parse_dates=['extrac_date'])


# encriptem els vitals

vitals['patient_deid'] = vitals['patient_ref'].apply(lambda x: xcc.encrypt_to_x_digits('number='+str(x),10, SALT))

# time shift utilitzant encriptat de patient_ref. Dintre dels temps de cada pacient mateix shift
vitals['weeks_to_add'] = vitals['patient_ref'].apply(lambda x: xcc.encrypt_to_x_digits('number='+str(x),3, SALT))
vitals['weeks_to_add'] = pd.to_timedelta(vitals['weeks_to_add'],unit='W')

# vitals te 1 dates: result_date
vitals['result_date'] = vitals['result_date']+vitals['weeks_to_add']


# eliminació de columnes innecessaries
vitals = vitals.drop(['patient_ref','patient_deid','weeks_to_add'],axis=1)


# tornem a posar els noms originals

#vitals = vitals.rename(columns={'patient_deid':'patient_ref','hosp_stay_deid':'hosp_stay_id','ward_stay_deid':'ward_stay_id','vitals_datetime':'m_events_datetime'})

# pujada de vitals a datanex

datanex.write_table_2(db='borrat_project', df=vitals, table_name='vitals_ds')






Database [borrat_project] session created...
<> Query Sucessful <>
Database [borrat_project] session created...
<> Table [vitals_ds] created <>


## Monitor Events Dictionary
Code block with commented example for creating a monitor events dictionary. Not executed in current workflow.

In [ ]:
"""# MONITOR_DIC creacio del diccionari de MONITOR_EVENTS


data = {
    "m_item_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    "Item": ["conc_state", "hr_ecg", "hr_osc", "rr_ip", "rr", "o2_sup", "pa_s", "presn", "pulsiox", "pulse", "temp_axi", "temp_bu", "temp_ct", "temp_tim"],
    "Description": ["Conscious state", "Heart rate measured by EKG", "Heart rate measured by oscillometry", "Respiratory rate measured by impedance pneumography",
                    "Respiratory rate measured by other means", "Oxygen administration nasal catheter", "Invasive systolic arterial pressure",
                    "Blood pressure (systolic and diastolic)", "Pulse oximetry monitoring", "Pulse rate", "Axillary temperature",
                    "Oral temperature", "Core temperature measurement", "Tympanic Temperature"],
    
    "SNOMED_Code": ["pending", 364075005, 364075005, 86290005, 86290005, 464225001, 251071003, 251070002, 284034009, 364094000, 415882003, 415945006, 386725007, 415974002]
}

# Diccionari per conc_state

1. Alert
2. Agitated
3. Confused
4. Responds to voice
5. Responds to pain
6. Unconscious



# Creating a DataFrame
monitor_dic = pd.DataFrame(data)


# pujada de monitor_dic a datanex

datanex.write_table(db='datascope4', df=monitor_dic, table_name='monitor_dic',schema='p_zero')"""

SyntaxError: invalid syntax (3007469307.py, line 17)

## Process Diagnostic Data
De-identify and process medical diagnosis data.

In [25]:
# DIAGNOSTICS tambe  DEID (encriptem dx) i time shift. 14 seg. 

# lectura de diagnostics de datanex_phi
diagnostics = datanex.query(db='borrat_project', query='select * from diagnostics')

# encritem els diagnostics patient_ref i hosp_stay_id

diagnostics['episode_ref'] = diagnostics['episode_ref'].apply(lambda x: xcc.encrypt_to_x_digits('number='+str(x),10, SALT))



# eliminació de columnes innecessaries
#diagnostics = diagnostics.drop(['patient_ref','hosp_stay_id','weeks_to_add'],axis=1)

# tornem a posar els noms originals

#diagnostics = diagnostics.rename(columns={'patient_deid':'patient_ref'})

# afegim columna index

diagnostics['diag_id'] = range(1, len(diagnostics) + 1)

# pujada de diagnostics a datanex
datanex.write_table_2(db='borrat_project', df=diagnostics, table_name='diagnostics_ds')





Database [borrat_project] session created...
<> Query Sucessful <>
Database [borrat_project] session created...
<> Table [diagnostics_ds] created <>


## Process Demographics Data
De-identify demographic data, removing direct identifiers while preserving necessary variables for analysis.

In [26]:
# DEMOG DEID
# deid i time shift de demog

# lectura de demog de datanex_phi
demog = datanex.query(db='borrat_project', query='select * from demographics')

# encriptem els demog patient_ref

demog['patient_ref'] = demog['patient_ref'].apply(lambda x: xcc.encrypt_to_x_digits('number='+str(x),10, SALT))

# eliminació de columnes innecessaries: eliminem data of birth perque 
# ja esta a ward_stays en forma de age_on_admission.

demog = demog.drop(['birth_date'],axis=1)


# tornem a posar els noms originals

#afegim columna index
demog['demog_id'] = range(1, len(demog) + 1)
# pujada de demog a datanex
datanex.write_table_2(db='borrat_project', df=demog, table_name='demographics_ds')


Database [borrat_project] session created...
<> Query Sucessful <>
Database [borrat_project] session created...
<> Table [demographics_ds] created <>
